Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking, CmdStan

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "05")
cd(ProjDir)

### snippet 5.1

In [3]:
wd = CSV.read(rel_path("..", "data", "WaffleDivorce.csv"), delim=';')
df = convert(DataFrame, wd);

mean_ma = mean(df[!, :Marriage])
df[!, :Marriage_s] = convert(Vector{Float64},
  (df[!, :Marriage]) .- mean_ma)/std(df[!, :Marriage]);

mean_mam = mean(df[!, :MedianAgeMarriage])
df[!, :MedianAgeMarriage_s] = convert(Vector{Float64},
  (df[!, :MedianAgeMarriage]) .- mean_mam)/std(df[!, :MedianAgeMarriage]);

df[1:6, [1, 7, 14, 15]]

rethinking_data = "
    Location Divorce  Marriage.s MedianAgeMarriage.s
1    Alabama    12.7  0.02264406          -0.6062895
2     Alaska    12.5  1.54980162          -0.6866993
3    Arizona    10.8  0.04897436          -0.2042408
4   Arkansas    13.5  1.65512283          -1.4103870
5 California     8.0 -0.26698927           0.5998567
"

"\n    Location Divorce  Marriage.s MedianAgeMarriage.s\n1    Alabama    12.7  0.02264406          -0.6062895\n2     Alaska    12.5  1.54980162          -0.6866993\n3    Arizona    10.8  0.04897436          -0.2042408\n4   Arkansas    13.5  1.65512283          -1.4103870\n5 California     8.0 -0.26698927           0.5998567\n"

Define the Stan language model

In [4]:
m5_3 = "
data {
  int N;
  vector[N] divorce;
  vector[N] marriage_z;
  vector[N] median_age_z;
}
parameters {
  real a;
  real bA;
  real bM;
  real<lower=0> sigma;
}
model {
  vector[N] mu = a + median_age_z * bA + marriage_z * bM;
  sigma ~ uniform( 0 , 10 );
  bA ~ normal( 0 , 1 );
  bM ~ normal( 0 , 1 );
  a ~ normal( 10 , 10 );
  divorce ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m5_3",
monitors = ["a", "bA", "bM", "sigma", "Divorce"],
 model=m5_3, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/05/tmp/m5_3.stan will be updated.



Input data for cmdstan

In [6]:
m5_3_data = Dict("N" => size(df, 1), "divorce" => df[!, :Divorce],
    "marriage_z" => df[!, :Marriage_s], "median_age_z" => df[!, :MedianAgeMarriage_s]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m5_3_data, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/05/tmp/m5_3.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Inference for Stan model: m5_3_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.100, 0.080, 0.11, 0.086) seconds, 0.37 seconds total
Sampling took (0.089, 0.097, 0.096, 0.10) seconds, 0.38 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%    N_Eff  N_Eff/s    R_hat
lp__              -46  3.6e-02     1.5    -49    -45    -44  1.6e+03  4.3e+03  1.0e+00
accept_stat__    0.92  1.5e-03   0.097  

Describe the draws

In [8]:
MCMCChains.describe(chn)

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean      │ std      │ naive_se   │ mcse       │ ess     │
│     │ Symbol     │ Float64   │ Float64  │ Float64    │ Float64    │ Any     │
├─────┼────────────┼───────────┼──────────┼────────────┼────────────┼─────────┤
│ 1   │ a          │ 9.68713   │ 0.218317 │ 0.0034519  │ 0.00312254 │ 3712.21 │
│ 2   │ bA         │ -1.12121  │ 0.295868 │ 0.00467809 │ 0.00472059 │ 2958.23 │
│ 3   │ bM         │ -0.126055 │ 0.296542 │ 0.00468874 │ 0.00487171 │ 2942.8  │
│ 4   │ sigma      │ 1.5218    │ 0.160265 │ 0.00253402 │ 0.00334189 │ 3360.38 │

Quantiles

│ Row │ parameters │ 2.5%      │ 25.0%     │ 50.0%    │ 75.0%     │ 97.5%     │
│     │ Symbol     │ Float64   │ Float64   │ Float64  │ Float64   │ Float64   │
├─────┼────────────┼───────────┼───────────┼──────────┼───────────┼───────────┤
│ 1   │ a          │ 9.2438    │ 9.54255   │ 9.68675  │ 9.83089   │ 10.1119   │
│ 2   │ bA         │ -

Rethinking results

In [9]:
rethinking_results = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.69 0.22  9.34 10.03  1313    1
bR    -0.12 0.30 -0.60  0.35   932    1
bA    -1.13 0.29 -1.56 -0.67   994    1
sigma  1.53 0.16  1.28  1.80  1121    1
"

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.69 0.22  9.34 10.03  1313    1\nbR    -0.12 0.30 -0.60  0.35   932    1\nbA    -1.13 0.29 -1.56 -0.67   994    1\nsigma  1.53 0.16  1.28  1.80  1121    1\n"

End of `05/5.3s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*